# Mimic of the DES night summary for Rubin Observatory for {{ params.day_obs }} of baseline simulation {{ params.sim_version }}

In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2024-06-18"
instrument = 'lsstcomcamsim'

## Introduction

This notebook is a mock-up of a night report for Rubin Observatory/LSST, modeled on the DES night summaries generated by the DES `nightsum` tool.
In this report, the LSST DDF fields are taken to be analogous to the DES SN fields, while all other fields are taken to be analogous to the DES wide survey fields.

I intend it as as a tool for test and exploration night reporting infrastructure, particularly the use of [Times Square](https://usdf-rsp-dev.slac.stanford.edu/times-square) in conjunction with other tools such as [`schedview`](https://schedview.lsst.io/).
I do *not* expected it to develop into "the" primary general night reporting tool for Rubin Observatory, but it *may* develop into a night report tool specialized for scheduler, strategy, and survey progress concerns, and I hope that it will be a helpful example for developers of other night reporting tools.

Interested users may check the notebook out of github [here](https://github.com/lsst/schedview_notebooks), install its [dependencies](https://schedview.lsst.io/installation.html), and produce the report with more flexibility (but less convenience) than offered by Times Square. For example, when run from Times Square this notebook only supports running data from the recent baseline opsim simulations, while it should be straightforward to modify this notebook in jupyter to run using bespoke simulations.

In [ ]:
# Uncomment for development
# %load_ext lab_black

In [ ]:
# Validate the input
import re

assert re.match(r"^\d\d\d\d-\d\d-\d\d$", day_obs) is not None
assert instrument in ('latiss', 'lsstcomcam', 'lsstcomcamsim')

In [ ]:
import datetime
import logging
import os
import sqlite3
import sys
import warnings

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlalchemy
from erfa import ErfaWarning
from functools import partial
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display
from lsst.resources import ResourcePath

In [ ]:
try:
    from lsst.summit.utils import ConsDbClient
    have_consdb = True
except ImportError:
    have_consdb = False
    import psycopg2

In [ ]:
try:
    del os.environ['http_proxy']
except KeyError:
    pass

try:
    del os.environ['HTTP_PROXY']
except KeyError:
    pass

In [ ]:
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/user/neilsen/data/rubin_sim_data"

In [ ]:
sys.path.insert(
    0, "/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages"
)

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(
        0,
        "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography",
    )
    sys.path.insert(
        0,
        "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/rubin_scheduler",
    )
    sys.path.insert(
        0,
        "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/rubin_sim",
    )
    sys.path.insert(
        0,
#        "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/schedview",
        "/sdf/data/rubin/user/neilsen/devel/schedview",
    )

In [ ]:
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.astro
import schedview.compute.visits
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import uranography
from rubin_scheduler.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Don't even complain.
warnings.filterwarnings(
    "ignore",
    category=astropy.utils.exceptions.AstropyWarning,
    message="Tried to get polar motions for times after IERS data is valid. Defaulting to polar motion from the 50-yr mean for those. This may affect precision at the arcsec level. Please check your astropy.utils.iers.conf.iers_auto_url and point it to a newer version if necessary.",
)

In [ ]:
# In simulations, we go far enough into the future that the erfa module finds it "dubious".
# Keep the complaints quiet.
warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*dubious year.*",
)

In [ ]:
# Don't complain about working with daytime MJDs either.
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.skybrightness_pre.sky_model_pre",
    category=UserWarning,
    message="Requested MJD between sunrise and sunset, returning closest maps",
)

In [ ]:
# Quiet unimportant chatter from healpy.
healpy_logger = logging.getLogger("healpy")
healpy_logger.setLevel(logging.WARNING)

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
day_obs.replace('-', '')

In [ ]:
day_obs_mjd = int(Time(day_obs).mjd)
day_obs_int = int(day_obs.replace('-', ''))
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"
use_matplotlib = True

In [ ]:
visits_query = f'''
    SELECT * FROM cdb_{instrument}.exposure
        WHERE obs_start_mjd IS NOT NULL
        AND s_ra IS NOT NULL
        AND s_dec IS NOT NULL
        AND sky_rotation IS NOT NULL
        AND ((band IS NOT NULL) OR (physical_filter IS NOT NULL))
        AND day_obs = {day_obs_int}
'''

In [ ]:
if have_consdb:
    consdb = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
    print("Using ConsDbClient")
    visits = consdb.query(visits_query).to_pandas()
else:
    connection = sqlalchemy.create_engine('postgresql://usdf@usdf-summitdb.slac.stanford.edu:5432/exposurelog')
    print("Using postgresql by way of sqlalchemy")
    visits = pd.read_sql(visits_query, connection)

print(f"Read {len(visits)} visits")

In [ ]:
exposure_opsimdb_map = {
        'obs_start_mjd': 'observationStartMJD',
        'obs_start': 'start_date',
        's_ra': 'fieldRA',
        's_dec': 'fieldDec',
        'sky_rotation': 'rotSkyPos',
        'band': 'filter',
        'airmass': 'airmass',
        'altitude': 'altitude',
        'azimuth': 'azimuth',
        'exp_time': 'visitExposureTime'
    }

In [ ]:
visits.rename(exposure_opsimdb_map, axis='columns', inplace=True)

In [ ]:
# visits.describe(include='all').T

In [ ]:
missing_filter = visits['filter'].isna()
visits.loc[missing_filter, 'filter'] = visits.loc[missing_filter, 'physical_filter'].str.get(0)

## Night narrative

Human written summaries and narratives describing the night and any problems or engineering tests that occurred during it will be an essential part of night reports (as it was in the DES night summaries on which this report is based).

Tools for recording such text and providing access to it are still under development (see the #rubinobs-nightlog channel in the LSSTC slack), and content not included here.
But, if it could be reported now, here is where it would be.

## Sun and Moon

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format="mjd").datetime
day_obs_date = datetime.date(
    day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day
)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

Modified Julian Date (MJD) is in units of days (UTC).

Local Sidereal Time (LST) is in units of degrees.

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc["night_middle", "MJD"]

In [ ]:
body_positions_wide = pd.DataFrame(
    model_observatory.almanac.get_sun_moon_positions(
        night_events.loc["night_middle", "MJD"]
    )
)
body_positions_wide.index.name = "r"
body_positions_wide.reset_index(inplace=True)

angle_columns = ["RA", "dec", "alt", "az"]
all_columns = angle_columns + ["phase"]
body_positions = (
    pd.wide_to_long(
        body_positions_wide,
        stubnames=("sun", "moon"),
        suffix=r".*",
        sep="_",
        i="r",
        j="coordinate",
    )
    .droplevel("r")
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

RA, dec, alt, and az are all in units of degrees.

Here, the "effective PSF FWHM" serves as a proxy for the measured PSF width in the DES `nightsum`, because it includes an estimates of all modeled contributions.

In an actual report using real data, the width of the PSF measured from the image should be plotted as well, so that differences can be easily noted.

## Visit map

In [ ]:
if len(visits):
    vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
        visits=visits,
        night_date=day_obs_date,
        timezone=timezone,
        observatory=observatory,
    )
    bokeh.io.show(vmap)
else:
    print("No visits")

The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The "Armillary sphere" shows the sphere in orthographic projection, with the center point of the projection controlled by the "center alt" and "center az" sliders beneath the plot. A static orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The "Planisphere" shows the sky in [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole, with R.A. increasing counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan. The fill colors are set according to the [descolors palette](https://github.com/DarkEnergySurvey/descolors):

 - <span style='background-color:#56b4e9'>&nbsp;&nbsp;&nbsp;</span><span style='color:#56b4e9'> blue</span>: u band
 - <span style='background-color:#008060'>&nbsp;&nbsp;&nbsp;</span><span style='color:#008060'> green</span>: g band
 - <span style='background-color:#ff4000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ff4000'> red</span>: r band
 - <span style='background-color:#850000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#850000'> brown/crimson</span>: i band
 - <span style='background-color:#6600cc'>&nbsp;&nbsp;&nbsp;</span><span style='color:#6600cc'> purple</span>: z band
 - <span style='background-color:#000000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#000000'> black</span>: y band

Both plots have the following additional annotations:
 - The gray background shows the planned final depth of the LSST survey.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) , completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.

## Table of exposures

In [ ]:
if len(visits):
    displayed_columns = [
        "start_date",
        "fieldRA",
        "fieldDec",
        "filter",
        "visitExposureTime",
#        "numExposures",
#        "t_eff",
#        "skyBrightness",
#        "seeingFwhmEff",
#        "cloud",
#        "note",
    ]
    displayed_visits_df = visits.loc[:, displayed_columns]
    with pd.option_context("display.max_rows", 2000):
        display(displayed_visits_df)
else:
    print("No visits")